# Data prepration

In [3]:
import numpy as np
import datetime
import os
import sklearn.metrics
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from scipy.stats import chi
from scipy import stats
from functions import mean_gen, varx_gen, vary_gen, spherical_data, discrete_angles

In [14]:
n = 200000


m_th = 40
m_ph = 40
thetas = discrete_angles(n, m_th, 2)
phis = discrete_angles(n, m_ph, 1)
name = ('discrete_model_mth%.0f_mph%.0f' %(m_th, m_ph))


# angles = np.array([5, 3])
# thetas, phis = angles[0] * np.ones(n), angles[1] * np.ones(n)
# name = ('discrete_model_th%.0f_ph%.0f' %(angles[0], angles[1]))

x_train, x_test, y_train, y_test = spherical_data(n, thetas, phis)

print(x_train)

[[ 1.42592595  0.2724874   4.77794461  2.85009733]
 [ 1.59258943 -0.40037405  6.15449331  2.50244622]
 [ 0.54891526 -3.19813743  4.9402326   0.73993036]
 ...
 [ 1.14933389  0.91900871  2.47385036  0.84497032]
 [ 0.49563322 -1.67486034  3.69017192  1.09744416]
 [-0.94236525  0.14972008  6.07990128  2.37616593]]


# Training Interpolated Model

In [15]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
]);

loss_fn = tf.keras.losses.BinaryCrossentropy()
model.compile(
    optimizer='adam',
    loss=loss_fn,
    metrics=['accuracy']
)

batch_size = 1000

checkpoint_path = "tutorial/checkpoints/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    verbose=1,
    save_freq=10*batch_size
)

log_dir = "tutorial/logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

EPOCHS=20
model.fit(
    x_train, y_train, epochs=EPOCHS,
    batch_size=batch_size,
    callbacks=[cp_callback, tensorboard_callback]
);

model.save('3dmodels/' + name)

2022-08-10 13:09:29.886057: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-08-10 13:09:29.886104: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2022-08-10 13:09:29.886260: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.


Epoch 1/20
 23/320 [=>............................] - ETA: 2s - loss: 0.6740 - accuracy: 0.5717

2022-08-10 13:09:30.399782: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-08-10 13:09:30.399815: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2022-08-10 13:09:30.423170: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2022-08-10 13:09:30.424145: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2022-08-10 13:09:30.436511: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: tutorial/logs/20220810-130929/train/plugins/profile/2022_08_10_13_09_30

2022-08-10 13:09:30.442111: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to tutorial/logs/20220810-130929/train/plugins/profile/2022_08_10_13_09_30/jupyter-matan.trace.json.gz
2022-08-10 13:09:30.444111: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: tutorial/logs/20220810-130

320/320 [==============================] - 2s 5ms/step - loss: 0.6498 - accuracy: 0.6043
Epoch 2/20
320/320 [==============================] - 2s 5ms/step - loss: 0.6386 - accuracy: 0.6178
Epoch 3/20
320/320 [==============================] - 2s 5ms/step - loss: 0.6349 - accuracy: 0.6230
Epoch 4/20
320/320 [==============================] - 2s 5ms/step - loss: 0.6326 - accuracy: 0.6253
Epoch 5/20
320/320 [==============================] - 2s 5ms/step - loss: 0.6314 - accuracy: 0.6270
Epoch 6/20
320/320 [==============================] - 2s 5ms/step - loss: 0.6304 - accuracy: 0.6280
Epoch 7/20
320/320 [==============================] - 1s 5ms/step - loss: 0.6296 - accuracy: 0.6287
Epoch 8/20
320/320 [==============================] - 2s 5ms/step - loss: 0.6291 - accuracy: 0.6285
Epoch 9/20
320/320 [==============================] - 2s 5ms/step - loss: 0.6287 - accuracy: 0.6292
Epoch 10/20
320/320 [==============================] - 2s 5ms/step - loss: 0.6283 - accuracy: 0.6301
Epoch 11/2

In [16]:
model = tf.keras.models.load_model('3dmodels/' + name)